1. Configuración Inicial del Proyecto:
- Crea un repositorio en GitHub para tu proyecto.
- Configura dos ramas en tu repositorio: main y development.
- Agrega un archivo README.md con una descripción del proyecto, instrucciones de instalación y uso.
2. **Carga y Preprocesamiento de Datos:**
- Carga los datos del archivo CSV utilizando NumPy.
- Realiza un preprocesamiento básico para asegurarte de que los datos estén limpios y listos para su análisis.
3. **Exploración de Datos:**
- Calcula el total de ventas por categoría de producto.
- Calcula el promedio de ventas diarias por categoría de producto.
- Identifica las categorías de productos con mayores y menores ventas.
4. **Manipulación de Datos:**
- Filtra los datos para mostrar solo las ventas de una categoría de producto específica.
- Realiza operaciones de suma, resta, multiplicación y división en los datos para obtener estadísticas adicionales.

# **1- Preprocesamiento Básico con numpy**

##**a) Leer y procesar los datos**

In [5]:
import numpy as np

# Importa la librería NumPy, útil para trabajar con datos numéricos y matrices.
# Carga los datos de un archivo CSV.
# - 'delimiter' indica que las columnas están separadas por comas.
# - 'dtype=None' permite que NumPy detecte automáticamente los tipos de datos.
# - 'encoding' asegura que se lea correctamente el archivo, incluso con caracteres especiales.
# - 'names=True' indica que la primera fila contiene los nombres de las columnas.
data_csv = np.genfromtxt('/content/retail_sales_dataset.csv', delimiter=',', dtype=None, encoding='utf-8', names=True)

# Muestra los nombres de las columnas del archivo CSV.
print("Nombres de las columnas:", data_csv.dtype.names, "\n")

# Imprime las primeras 5 filas de los datos cargados.
print(data_csv[:5])

Nombres de las columnas: ('Transaction_ID', 'Date', 'Customer_ID', 'Gender', 'Age', 'Product_Category', 'Quantity', 'Price_per_Unit', 'Total_Amount') 

[(1, '2023-11-24', 'CUST001', 'Male', 34, 'Beauty', 3,  50,  150)
 (2, '2023-02-27', 'CUST002', 'Female', 26, 'Clothing', 2, 500, 1000)
 (3, '2023-01-13', 'CUST003', 'Male', 50, 'Electronics', 1,  30,   30)
 (4, '2023-05-21', 'CUST004', 'Male', 37, 'Clothing', 1, 500,  500)
 (5, '2023-05-06', 'CUST005', 'Male', 30, 'Beauty', 2,  50,  100)]


## **b) Verificar si hay valores nulos y reemplazarlos**

In [2]:
# Recorre cada columna en los datos cargados, utilizando los nombres de las columnas.
for column in data_csv.dtype.names:
    # Verifica si la columna tiene datos de tipo float (números con decimales).
    if data_csv[column].dtype == 'float':
        # Cuenta cuántos valores nulos (NaN) hay en la columna y los imprime.
        print(f"\nValores nulos en columna {column}:", np.sum(np.isnan(data_csv[column])))
    else:
        # Si la columna no es de tipo float, asume que no tiene valores nulos (NaN).
        print(f'No existen datos nulos en la columna {column}')

No existen datos nulos en la columna Transaction_ID
No existen datos nulos en la columna Date
No existen datos nulos en la columna Customer_ID
No existen datos nulos en la columna Gender
No existen datos nulos en la columna Age
No existen datos nulos en la columna Product_Category
No existen datos nulos en la columna Quantity
No existen datos nulos en la columna Price_per_Unit
No existen datos nulos en la columna Total_Amount


Cómo se eligió el dataset directo de Kagle, no se encontraron datos nulos o faltantes en el dataset, supongo que, dependiendo del objetivo, si se debería revisar si hay datos repetidos

# **2. Exploración de Datos**

## **a) Calcular el total de ventas por categoría de producto**

In [17]:
# Obtener los valores únicos y su cantidad en la columna 'Product_Category'
unique_values, counts = np.unique(data_csv['Product_Category'], return_counts=True)

# Mostrar los valores únicos
print("\nValores únicos en 'Product_Category':", unique_values)

# Mostrar la cantidad total de valores únicos
print("\nCantidad de valores únicos:", len(unique_values))

# Mostrar las frecuencias de cada valor único
print("\nFrecuencias de los valores únicos:", dict(zip(unique_values, counts)))
#dict(zip(claves,valors)): para combinar los valores únicos y sus frecuencias en un diccionario para fácil lectura.


Valores únicos en 'Product_Category': ['Beauty' 'Clothing' 'Electronics']

Cantidad de valores únicos: 3

Frecuencias de los valores únicos: {'Beauty': 307, 'Clothing': 351, 'Electronics': 342}


In [ ]:
# Utiliza np.unique para obtener las categorías únicas de 'Product_Category' y el índice correspondiente.
# 'return_inverse=True' devuelve el arreglo 'inverse' con índices que corresponden a las categorías.
categories, inverse = np.unique(data_csv['Product_Category'], return_inverse=True)

# Utiliza np.bincount para contar y sumar los valores de 'Transaction_ID' en función de los índices en 'inverse'.
# 'weights=data_csv['Transaction_ID']' asegura que cada valor en 'Transaction_ID' se sume en lugar de simplemente contar las ocurrencias.
totals = np.bincount(inverse, weights=data_csv['Transaction_ID'])

# Crea un diccionario 'total_ventas_por_categoria_3' que mapea las categorías a sus respectivos totales de ventas.
total_ventas_por_categoria_3 = dict(zip(categories, totals))

# Imprime el total de ventas por categoría.
print("\nTotal de ventas por categoría:")
print(total_ventas_por_categoria_3)


Total de ventas por categoría:
{'Beauty': 150864.0, 'Clothing': 173725.0, 'Electronics': 175911.0}


## **b) Calcular el promedio de ventas diarias por categoría de producto**

In [25]:
# Obtiene las fechas únicas de la columna 'Date'.
dates = np.unique(data_csv['Date'])

# Inicializa un diccionario vacío para almacenar las ventas diarias por categoría.
ventas_diarias = {}

# Itera sobre cada categoría única de 'Product_Category'.
for category in categories:
    # Obtiene las ventas correspondientes a la categoría actual.
    ventas_por_categoria = data_csv['Total_Amount'][data_csv['Product_Category'] == category]

    # Obtiene las fechas correspondientes a la categoría actual.
    fechas_categoria = data_csv['Date'][data_csv['Product_Category'] == category]

    # Para cada fecha única, calcula la suma de las ventas en esa fecha para la categoría actual.
    ventas_diarias[category] = {fecha: np.sum(ventas_por_categoria[fechas_categoria == fecha]) for fecha in dates}

# Calcula el promedio de ventas diarias por categoría, promediando los valores de cada diccionario de ventas diarias.
promedio_ventas_diarias = {category: round(np.mean(list(d.values())),3) for category, d in ventas_diarias.items()}

# Imprime el promedio de ventas diarias por categoría.
print("\nPromedio de ventas diarias por categoría:")
print(promedio_ventas_diarias)


Promedio de ventas diarias por categoría:
{'Beauty': 415.986, 'Clothing': 450.957, 'Electronics': 454.797}


## **c) Identificar las categorías con mayores y menores ventas**

In [27]:
categories_array = np.array(list(total_ventas_por_categoria.keys()))
ventas_array = np.array(list(total_ventas_por_categoria.values()))

indice_max = np.argmax(ventas_array)
indice_min = np.argmin(ventas_array)

categoria_max_ventas = categories_array[indice_max]
categoria_min_ventas = categories_array[indice_min]
print(f"\nCategoría con mayores ventas: {categoria_max_ventas}")
print(f"\nCategoría con menores ventas: {categoria_min_ventas}")


Categoría con mayores ventas: Electronics

Categoría con menores ventas: Beauty


# **3. Manipulación de Datos**


## **a) Filtrar los datos por una categoría de producto específica**


### **a.1) Uso de máscaras booleanas**

In [ ]:
# Crea una máscara booleana que selecciona solo las filas correspondientes a la categoría 'Electronics'.
mascara = data_csv['Product_Category'] == 'Electronics'

# Filtra el DataFrame usando la máscara.
categoria_filtrada = data_csv[mascara]

# Imprime los primeros 5 registros de la categoría filtrada.
print("\nDatos filtrados para la categoría 'Electronics':")
print(categoria_filtrada[:5])


Datos filtrados para la categoría 'Electronics':
[( 3, '2023-01-13', 'CUST003', 'Male', 50, 'Electronics', 1,  30,   30)
 ( 8, '2023-02-22', 'CUST008', 'Male', 30, 'Electronics', 4,  25,  100)
 ( 9, '2023-12-13', 'CUST009', 'Male', 63, 'Electronics', 2, 300,  600)
 (13, '2023-08-05', 'CUST013', 'Male', 22, 'Electronics', 3, 500, 1500)
 (15, '2023-01-16', 'CUST015', 'Female', 42, 'Electronics', 4, 500, 2000)]


###**a.2) Uso de índices obtenidos con np.where**

In [ ]:
# Usa np.where para encontrar los índices de las filas donde la categoría es 'Clothing'
indices = np.where(data_csv['Product_Category'] == 'Clothing')[0]

# Filtra el DataFrame usando los índices encontrados
categoria_filtrada = data_csv[indices]

# Imprime los primeros 5 registros de la categoría filtrada
print("\nDatos filtrados para la categoría 'Clothing':")
print(categoria_filtrada[:5])


Datos filtrados para la categoría 'Clothing':
[( 2, '2023-02-27', 'CUST002', 'Female', 26, 'Clothing', 2, 500, 1000)
 ( 4, '2023-05-21', 'CUST004', 'Male', 37, 'Clothing', 1, 500,  500)
 ( 7, '2023-03-13', 'CUST007', 'Male', 46, 'Clothing', 2,  25,   50)
 (10, '2023-10-07', 'CUST010', 'Female', 52, 'Clothing', 4,  50,  200)
 (11, '2023-02-14', 'CUST011', 'Male', 23, 'Clothing', 2,  50,  100)]


## **b) Realizar operaciones matemáticas sobre los datos**

### **b.1) Usando máscaras booleanas directamente**

In [ ]:
# Calcula el total de ventas para la categoría 'Clothing'
ventas_categoria_A = np.sum(data_csv['Total_Amount'][data_csv['Product_Category'] == 'Clothing'])

# Calcula el total de ventas para la categoría 'Electronics'
ventas_categoria_B = np.sum(data_csv['Total_Amount'][data_csv['Product_Category'] == 'Electronics'])

# Suma las ventas de ambas categorías
suma_ventas = ventas_categoria_A + ventas_categoria_B

# Resta las ventas de 'Electronics' de las ventas de 'Clothing'
resta_ventas = ventas_categoria_A - ventas_categoria_B

# Multiplica las ventas de 'Clothing' por 1.1
multiplicacion_ventas = ventas_categoria_A * 1.1

# Divide las ventas de 'Clothing' entre las de 'Electronics', asegurándose de no dividir por 0
division_ventas = ventas_categoria_A / ventas_categoria_B if ventas_categoria_B != 0 else 0

# Imprime los resultados de las operaciones
print(f"\nSuma de ventas entre Clothing y Electronics: {suma_ventas}")
print(f"Resta de ventas entre Clothing y Electronics: {resta_ventas}")
print(f"Multiplicación de ventas de Clothing por 1.1: {multiplicacion_ventas}")
print(f"División de ventas entre Clothing y Electronics: {division_ventas}")


Suma de ventas entre Clothing y Electronics: 312485
Resta de ventas entre Clothing y Electronics: -1325
Multiplicación de ventas de Clothing por 1.1: 171138.0
División de ventas entre Clothing y Electronics: 0.9915553997641885


### **b.2) Usando índices con np.where y operaciones numpy**

In [ ]:
# Creamos un arreglo con las categorías de productos
categorias = np.array(data_csv['Product_Category'])

# Creamos un arreglo con el total de las ventas
ventas_totales = np.array(data_csv['Total_Amount'])

# Calculamos el total de ventas para la categoría 'Clothing'
ventas_categoria_A = np.sum(ventas_totales[categorias == 'Clothing'])

# Calculamos el total de ventas para la categoría 'Electronics'
ventas_categoria_B = np.sum(ventas_totales[categorias == 'Electronics'])

# Sumamos las ventas de las dos categorías
suma_ventas = np.add(ventas_categoria_A, ventas_categoria_B)

# Restamos las ventas de 'Electronics' de las de 'Clothing'
resta_ventas = np.subtract(ventas_categoria_A, ventas_categoria_B)

# Multiplicamos las ventas de 'Clothing' por 1.1 (por ejemplo, un incremento del 10%)
multiplicacion_ventas = np.multiply(ventas_categoria_A, 1.1)

# Dividimos las ventas de 'Clothing' entre las de 'Electronics', asegurándonos de que 'Electronics' no tenga ventas en 0
division_ventas = np.divide(ventas_categoria_A, ventas_categoria_B) if ventas_categoria_B != 0 else 0

# Imprimimos los resultados de las operaciones realizadas
print(f"\nSuma de ventas entre Clothing y Electronics: {suma_ventas}")
print(f"Resta de ventas entre Clothing y Electronics: {resta_ventas}")
print(f"Multiplicación de ventas de Clothing por 1.1: {multiplicacion_ventas}")
print(f"División de ventas entre Clothing y Electronics: {division_ventas}")


Suma de ventas entre Clothing y Electronics: 312485
Resta de ventas entre Clothing y Electronics: -1325
Multiplicación de ventas de Clothing por 1.1: 171138.0
División de ventas entre Clothing y Electronics: 0.9915553997641885
